In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path = ""
df=pd.read_csv(path+'cleaned_train.csv')
val_df=pd.read_csv(path+'cleaned_validation.csv')


In [ ]:
val_df.head()

,id,feedback_text,lang,toxic,text
0,1203,İyi tamam olabilir. Balkanlar maddesini gelişt...,tr,0,Well okay. We improve the Balkans substance an...
1,5871,"Por dios, y la canción de John Lennon: http://...",es,1,"By God, and John Lennon's song: http://www.you..."
2,3590,Selam. Öncelikle tebrik ederim... Bu arada ken...,tr,0,"Hi. First of all, I congratulate you ... In th..."
3,447,Leggiti tutte le discussioni. Magari cancellal...,it,1,Read all the discussions. Maybe delete and get...
4,6634,A LAS TOKITAS NOS VALE QUE LAS JONATICAS INSUL...,es,1,To the tokitas it is worth that the Jonaticas ...


In [ ]:
df.shape

(23468, 9)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23468 entries, 0 to 23467
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             23468 non-null  object
 1   feedback_text  23468 non-null  object
 2   toxic          23468 non-null  int64 
 3   abusive        23468 non-null  int64 
 4   vulgar         23468 non-null  int64 
 5   menace         23468 non-null  int64 
 6   offense        23468 non-null  int64 
 7   bigotry        23468 non-null  int64 
 8   cleaned_text   23468 non-null  object
dtypes: int64(6), object(3)
memory usage: 1.6+ MB


## Transformer-Based Models: Fine-tune BERT or XLM for the task

In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

Prepare Data

In [ ]:
val_df.shape

(840, 5)

In [ ]:
from transformers import AutoTokenizer

df=df.iloc[:2000]
train_texts= df['cleaned_text'].astype(str).tolist()
train_labels= df['toxic'].tolist()

val_texts= val_df['text'].astype(str).tolist()
val_labels= val_df['toxic'].tolist()


tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")  # or 'xlm-roberta-base'

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
#pytorch dataset
import torch

class ToxicDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)

train_dataset = ToxicDataset(train_encodings, train_labels)
val_dataset = ToxicDataset(val_encodings, val_labels)

In [ ]:
#Load Pretrained Model and Training
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

trainer.train()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-08ba37a1e912>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wan

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:


Abort: 

Evaluation

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

In [ ]:

preds_output = trainer.predict(val_dataset)
preds = torch.argmax(torch.tensor(preds_output.predictions), axis=1)
print(classification_report(val_labels, preds))

Error: You must call wandb.init() before wandb.log()

In [ ]:
from imblearn.over_sampling import RandomOverSampler
import numpy as np

# Flatten padded sequence for oversampling
X_rs = X_train_padded.reshape((X_train_padded.shape[0], -1))

ros = RandomOverSampler(random_state=42)
X_oversampled, y_oversampled = ros.fit_resample(X_rs, y_train)

# Reshape back to original padded shape
X_oversampled = X_oversampled.reshape((-1, X_train_padded.shape[1]))

model.fit(X_oversampled, y_oversampled,
          epochs=5,
          batch_size=32,
          validation_data=(X_val_padded, y_val))

NameError: name 'X_train_padded' is not defined